<a href="https://colab.research.google.com/github/mtzig/LIDC_GDRO/blob/main/civil_comments_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi -L  

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-6bb9d08a-3739-a3af-4ab9-607a204e9175)


In [3]:
!git clone https://github.com/MsAsterisk1/ERM_GDRO.git
%cd /content/ERM_GDRO

fatal: destination path 'ERM_GDRO' already exists and is not an empty directory.
/content/ERM_GDRO


In [4]:
!git pull

Already up to date.


In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd
from utils.process_data_utils import  get_dataloaders, get_CivilComments_Datasets, get_CivilComments_df, split_dataset, get_partitioned_dataloader
from loss import ERMLoss, GDROLoss, ERMGDROLoss, UpweightLoss
from train_eval import train, evaluate, train_epochs
import torch
from models import BertClassifier
from math import ceil
import transformers

Now lets try getting Bert working

In [7]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [8]:
df = get_CivilComments_df()

In [9]:
datasets = get_CivilComments_Datasets(CC_df=df, device=DEVICE)

In [10]:
tr,cv,test = get_dataloaders(datasets, (16,32,32), split=True)

This is how we would initialize model

In [11]:
model = BertClassifier(device=DEVICE)
epochs = 5
# loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
loss_fn = ERMGDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 2, 0.5, partitioned=True)

num_training_steps = ceil(tr.batches_per_epoch() / 16) * epochs
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001, weight_decay=0.01)

scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

train_epochs(epochs, tr, cv, test, model, loss_fn, optimizer, scheduler=scheduler, vector_subclass=True, verbose=True, num_subclasses=18, save_weights_name='CC_ERM_GDRO', gradient_clip = 1)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

Epoch 1 / 5
Average training loss: 0.3155871058200269
For Cross Val:
Loss: 0.33714956045150757 Accuracy: 0.8526117751217352 
Accuracy over subgroups: [0.91048951 0.87418936 0.91340782 0.82552083 0.89453125 0.84567901
 0.94746717 0.92370892 0.87446708 0.77876543 0.80800781 0.53503185
 0.87926068 0.56508135 0.72694175 0.41823056 0.48337469 0.90742875] 
Worst Group Accuracy: 0.41823056300268097
Epoch 2 / 5
Average training loss: 0.3051491806305194
For Cross Val:
Loss: 0.33715304732322693 Accuracy: 0.8526117751217352 
Accuracy over subgroups: [0.91048951 0.87418936 0.91340782 0.82552083 0.89453125 0.84567901
 0.94746717 0.92370892 0.87446708 0.77876543 0.80800781 0.53503185
 0.87926068 0.56508135 0.72694175 0.41823056 0.48337469 0.90742875] 
Worst Group Accuracy: 0.41823056300268097
Epoch 3 / 5
Average training loss: 0.3078997809123641
For Cross Val:
Loss: 0.33714476227760315 Accuracy: 0.8526117751217352 
Accuracy over subgroups: [0.91048951 0.87418936 0.91340782 0.82552083 0.89453125 0.84

KeyboardInterrupt: ignored

In [12]:
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

Accuracy: 0.8494715283072461 
Accuracy over subgroups: [0.89151158 0.87092511 0.92269737 0.83492063 0.90350338 0.87307692
 0.94274561 0.93276937 0.87203718 0.76579557 0.8021017  0.51339564
 0.86348236 0.58786181 0.74630872 0.42968516 0.48174035 0.90470714] 
Worst Group Accuracy: 0.4296851574212894
